------------------------------------------------------------------------------------------------------------------------------------
Quantum Finance - Apresentação/Estudo de aplicação em Reinforcement Learning (Turma 8DTSR)
------------------------------------------------------------------------------------------------------------------------------------

- O que deve ser apresentado?

Ao final da disciplina deverá ser apresentado o planejamento de uma aplicação ou então pesquisa com relação a um problema em específico, se utilizando de Aprendizagem por Reforço (RL).

O problema a ser solucionado via Reinforcement Learning é o seguinte:

A QuantumFinance deseja criar um fundo automatizado para operar três ativos: Vale, Petrobrás e Brasil Foods. O objetivo é desenvolver e simular um agente de Reinforcement Learning (RL) capaz de tomar decisões financeiras, como compra, venda ou manutenção de posição, com base em dados históricos.

Tarefas:

- Definir o Problema de RL:
Identificar estados, ações e recompensas no contexto financeiro.

- Implementar ou apresentar exemplo de aplicação do Agente de RL:
Escolher um algoritmo de RL (ex.: Q-Learning ou DQN) e aplicá-lo para treinar o agente utilizando os dados históricos.

- Avaliação do Desempenho:
Descrever ou simular o comportamento do agente e, se possível, analisar os resultados com métricas financeiras relevantes (lucro, sharpe ratio, etc.).

- Entregáveis:
Relatório explicativo com desenho do agente, resultados obtidos e insights;
Se aplicável, código-fonte com a implementação e simulações realizadas.


As equipes devem atentar para os seguintes pontos:

Ingestão de dados - Quais seriam os dados a serem utilizados para resolver o problema em questão, conseguem mostrar exemplos de como estão dispostos esses dados de alguma forma? Há bases que podem ser utilizadas em um problema de RL?  

(Dica: pesquisar uma aplicação/problema em algum site de pesquisa de datasets como https://datasetsearch.research.google.com/, https://www.kaggle.com/ ou no https://paperswithcode.com/task/reinforcement-learning-1).

Processamento: Têm exemplos de como esse problema/aplicação tem sido implementado? Que tipo de algoritmo é mais utilizado? Tem como exemplificar o funcionamento de alguma forma? 

(Dica: tem muita coisa legal em repositórios no github como  o https://github.com/aikorea/awesome-rl).

Saída do processo: Qual a saída do processo? Qual o objetivo do agente no caso escolhido? Negociar um ativo ao melhor preço? Criar um documento com feedback humano para os analistas?

- Sugiro estruturar o trabalho tendo por base os seguintes tópicos:

Introdução e Problemática;
Motivação e Objetivo;
Apresentação da aplicação estudada/estrutura;
Considerações e Potencial.

- Avaliação:

Material escrito entregue (de preferência slides, vídeo com breve apresentação etc.);
Aderência aos pontos;
Avaliação de código/aplicação, se aplicável;
Originalidade, “fit” para com o perfil do curso/alunos e “doability”.

- Sites para pesquisas quanto a bases de dados:

http://archive.ics.uci.edu/ml/index.php
http://dados.gov.br/
https://www.kaggle.com/datasets
https://github.com/fivethirtyeight/data
http://www.portaldatransparencia.gov.br/
https://paperswithcode.com/
https://datasetsearch.research.google.com/

Mais sobre Aprendizagem por Reforço:

https://towardsdatascience.com/applications-of-reinforcement-learning-in-real-world-1a94955bcd12
https://wiki.pathmind.com/deep-reinforcement-learning#define

Entrega do trabalho: dia 23/02/2025

Quaisquer dúvidas é só me contactar: profahirton.lopes@fiap.com.br

[]'s

# Preparando dados

In [1]:
import pandas as pd
import numpy as np

import gymnasium as gym
from gymnasium import spaces

import random


In [2]:
df = pd.read_csv('b3.csv')

In [3]:
df.dtypes

codigo_registro                int64
data_pregao                   object
codigo_mercado                 int64
ativo                         object
codigo_bdi                     int64
nome                          object
especificador                 object
identificacao_mercado         object
moeda_cotacao                 object
preco_abertura               float64
preco_maximo                 float64
preco_minimo                 float64
preco_fechamento             float64
preco_melhor_compra          float64
preco_melhor_venda           float64
numero_negocios                int64
quantidade_acoes               int64
volume_financeiro            float64
preco_fechamento_ajustado    float64
fator_ajuste                   int64
codigo_papel_registro         object
dtype: object

In [4]:
df.head()

,codigo_registro,data_pregao,codigo_mercado,ativo,codigo_bdi,nome,especificador,identificacao_mercado,moeda_cotacao,preco_abertura,...,preco_minimo,preco_fechamento,preco_melhor_compra,preco_melhor_venda,numero_negocios,quantidade_acoes,volume_financeiro,preco_fechamento_ajustado,fator_ajuste,codigo_papel_registro
0,1,2022-02-03,2,VALE3,10,VALE,ON,NM,R$,86.30,...,85.32,86.01,85.75,85.74,0,85755787300000,191794000.0,16496500.39,0,0000000000009999123100000010000000000000BRVALE...
1,1,2022-01-03,2,VALE3,10,VALE,ON,NM,R$,79.00,...,78.00,78.76,78.00,78.00,0,78047400000000,185572000.0,14617464.93,0,0000000000009999123100000010000000000000BRVALE...
2,1,2022-02-03,2,PETR4,10,PETROBRAS,PN,N2,R$,32.35,...,31.62,32.03,32.07,32.07,0,32089236000000,510878000.0,16363981.67,0,0000000000009999123100000010000000000000BRPETR...
3,1,2022-02-03,2,BRFS3,10,BRF SA,ON,NM,R$,20.01,...,19.82,19.97,19.85,19.85,0,19864616200000,191324000.0,3821164.14,0,0000000000009999123100000010000000000000BRBRFS...
4,1,2022-01-03,2,PETR4,10,PETROBRAS,PN,N2,R$,28.54,...,28.53,28.99,29.09,29.08,0,29098247900000,527047000.0,15284293.87,0,0000000000009999123100000010000000000000BRPETR...


# Preaprando DataFrame apenas com as features de interesse

In [5]:
feats = [
    'data_pregao',
    'ativo',
    'preco_abertura',
    'preco_maximo',
    'preco_minimo',
    'preco_fechamento'
]
    
df = df[feats].copy()

In [6]:
df.head()

,data_pregao,ativo,preco_abertura,preco_maximo,preco_minimo,preco_fechamento
0,2022-02-03,VALE3,86.30,86.78,85.32,86.01
1,2022-01-03,VALE3,79.00,79.97,78.00,78.76
2,2022-02-03,PETR4,32.35,32.92,31.62,32.03
3,2022-02-03,BRFS3,20.01,20.30,19.82,19.97
4,2022-01-03,PETR4,28.54,29.22,28.53,28.99


# Reinforcement Learning

## Preparando ambiente

In [7]:
class TradingEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, data: pd.DataFrame, window_ma: int = 5, transaction_cost: float = 0.001):
        super().__init__()
        self.data = data.copy()  # DataFrame com índice de datas e colunas: 'VALE3', 'PETR4', 'BRFS3'
        self.dates = self.data.index.tolist()
        self.assets = list(self.data.columns)
        self.n_assets = len(self.assets)
        self.window_ma = window_ma
        self.transaction_cost = transaction_cost
        
        # Calcula a média móvel para cada ativo
        self.ma_data = self.data.rolling(window=self.window_ma, min_periods=1).mean()
        
        # Inicia após ter dados suficientes para MA
        self.current_step = self.window_ma  
        
        # Carteira inicial
        self.initial_cash = 10000.0
        self.cash = self.initial_cash
        self.positions = {asset: 0 for asset in self.assets}
        
        # Espaço de observação: para cada ativo: [diff (0,1,2), indicador (0,1), posição (0,1)]
        low = np.array([0] * (self.n_assets * 3))
        high = np.array([2, 1, 1] * self.n_assets)
        self.observation_space = spaces.Box(low=low, high=high, dtype=np.int32)
        
        # Espaço de ação: para cada ativo: 0 = hold, 1 = buy, 2 = sell
        self.action_space = spaces.MultiDiscrete([3] * self.n_assets)

    def reset(self, seed=None, options=None):
        self.current_step = self.window_ma
        self.cash = self.initial_cash
        self.positions = {asset: 0 for asset in self.assets}
        return self._get_obs(), {}

    def _get_obs(self):
        obs = []
        for asset in self.assets:
            prev_price = self.data.iloc[self.current_step - 1][asset]
            curr_price = self.data.iloc[self.current_step][asset]
            # Diferença discretizada:
            if curr_price > prev_price:
                diff = 2
            elif curr_price < prev_price:
                diff = 0
            else:
                diff = 1
            # Indicador: preço acima da média móvel?
            ma = self.ma_data.iloc[self.current_step][asset]
            above_ma = 1 if curr_price > ma else 0
            pos = self.positions[asset]
            obs.extend([diff, above_ma, pos])
        return np.array(obs, dtype=np.int32)

    def step(self, action):
        current_prices = {asset: self.data.iloc[self.current_step][asset] for asset in self.assets}
        
        # Executa a ação para cada ativo com custo de transação
        for i, asset in enumerate(self.assets):
            act = action[i]
            price = current_prices[asset]
            cost = price * self.transaction_cost
            if act == 1:  # Buy
                if self.positions[asset] == 0 and self.cash >= (price + cost):
                    self.positions[asset] = 1
                    self.cash -= (price + cost)
            elif act == 2:  # Sell
                if self.positions[asset] == 1:
                    self.positions[asset] = 0
                    self.cash += (price - cost)
            # Se hold, nada é feito

        portfolio_value = self.cash + sum(self.positions[asset] * current_prices[asset] for asset in self.assets)
        
        self.current_step += 1
        done = self.current_step >= len(self.data)
        
        if not done:
            next_prices = {asset: self.data.iloc[self.current_step][asset] for asset in self.assets}
        else:
            next_prices = current_prices
        
        new_portfolio_value = self.cash + sum(self.positions[asset] * next_prices[asset] for asset in self.assets)
        reward = new_portfolio_value - portfolio_value
        
        obs = self._get_obs() if not done else np.zeros(self.observation_space.shape, dtype=np.int32)
        info = {"portfolio_value": new_portfolio_value}
        
        return obs, reward, done, False, info

    def render(self):
        print(f"Step: {self.current_step}, Cash: {self.cash:.2f}, Positions: {self.positions}")


## Funções para mapeamento de estado/ação

Converter vetor de estado para índice.

O estado tem 9 features (3 por ativo). Para manter a abordagem tabular, precisamos discretizar ou codificar cada feature.

Aqui, assumimos que:
- Para cada ativo, diff: {0,1,2}  (3 valores)
- indicador: {0,1}                (2 valores)
- posição: {0,1}                  (2 valores)

Portanto, cada ativo tem 3*2*2 = 12 combinações e para 3 ativos, total = 12^3 = 1728 estados.


In [8]:
def state_to_index(state):
    idx = 0
    multiplier = 1
    for i in range(0, len(state), 3):
        diff, ind, pos = state[i], state[i+1], state[i+2]
        substate = diff + ind * 3 + pos * 6
        idx += substate * multiplier
        multiplier *= 12
    return idx

def action_to_index(action):
    a0, a1, a2 = action
    return a0 + a1 * 3 + a2 * 9

def index_to_action(action_index):
    a0 = action_index % 3
    a1 = (action_index // 3) % 3
    a2 = (action_index // 9) % 3
    return np.array([a0, a1, a2])

In [9]:
n_states = 1728   # 12^3 estados
n_actions = 27    # 3^3 ações
alpha = 0.1
gamma = 0.95
epsilon = 1.0
min_epsilon = 0.01
epsilon_decay = 0.995

In [10]:
Q_table = np.random.uniform(low=-0.01, high=0.01, size=(n_states, n_actions))
n_episodes = 2500

In [11]:
df.head()

,data_pregao,ativo,preco_abertura,preco_maximo,preco_minimo,preco_fechamento
0,2022-02-03,VALE3,86.30,86.78,85.32,86.01
1,2022-01-03,VALE3,79.00,79.97,78.00,78.76
2,2022-02-03,PETR4,32.35,32.92,31.62,32.03
3,2022-02-03,BRFS3,20.01,20.30,19.82,19.97
4,2022-01-03,PETR4,28.54,29.22,28.53,28.99


## "Girando" o DataFrame

In [12]:
df['data_pregao'] = pd.to_datetime(df['data_pregao'])
df_prices = df.pivot(index='data_pregao', columns='ativo', values='preco_fechamento')
df_prices = df_prices.sort_index()
df_prices.head()

ativo,BRFS3,PETR4,VALE3
data_pregao,,,
2022-01-03,22.82,28.99,78.76
2022-01-04,22.40,29.18,77.65
2022-01-05,23.14,28.52,77.86
2022-01-06,23.60,28.21,79.62
2022-01-07,24.42,28.07,82.16


In [13]:
env = TradingEnv(df_prices, window_ma=5, transaction_cost=0.001)

In [ ]:
for episode in range(n_episodes):
    state, _ = env.reset()
    state_idx = state_to_index(state)
    done = False
    total_reward = 0.0

    while not done:
        if random.uniform(0, 1) < epsilon:
            action_idx = random.randint(0, n_actions - 1)
        else:
            action_idx = np.argmax(Q_table[state_idx, :])
        action = index_to_action(action_idx)
        
        next_state, reward, done, truncated, info = env.step(action)
        total_reward += reward
        next_state_idx = state_to_index(next_state) if not done else 0
        
        best_next_action = np.argmax(Q_table[next_state_idx, :])
        Q_table[state_idx, action_idx] += alpha * (reward + gamma * Q_table[next_state_idx, best_next_action] - Q_table[state_idx, action_idx])
        state_idx = next_state_idx

    epsilon = max(min_epsilon, epsilon * epsilon_decay)
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode+1}/{n_episodes} - Total Reward: {total_reward:.2f} - Epsilon: {epsilon:.3f}")

print("Treinamento concluído!")

# Simulação de teste

In [32]:
epsilon_test = 0.05  # Permite que, ocasionalmente, uma ação diferente seja escolhida

state, _ = env.reset()
done = False
portfolio_values = [env.cash + sum(env.positions[asset] * env.data.iloc[env.current_step][asset] for asset in env.assets)]
daily_returns = []
trade_log = []  # Para registrar operações
step_counter = 0

while not done:
    step_counter += 1
    state_idx = state_to_index(state)
    # Política epsilon-greedy com epsilon_test
    if random.uniform(0, 1) < epsilon_test:
        action_idx = random.randint(0, n_actions - 1)
    else:
        action_idx = np.argmax(Q_table[state_idx, :])
    action = index_to_action(action_idx)
    
    # Registro de trades (se não hold)
    for i, act in enumerate(action):
        if act != 0:
            trade_log.append({
                "step": step_counter,
                "asset": env.assets[i],
                "action": "Buy" if act == 1 else "Sell",
                "price": env.data.iloc[env.current_step][env.assets[i]],
                "cash_before": env.cash,
                "positions_before": env.positions.copy()
            })
    
    next_state, reward, done, truncated, info = env.step(action)
    
    current_value = info["portfolio_value"]
    portfolio_values.append(current_value)
    daily_returns.append(reward / (portfolio_values[-2] + 1e-8))
    state = next_state

final_value = portfolio_values[-1]
profit = final_value - env.initial_cash
if np.std(daily_returns) > 0:
    sharpe_ratio = np.mean(daily_returns) / np.std(daily_returns) * np.sqrt(252)
else:
    sharpe_ratio = 0.0

print(f"Valor final da carteira: {final_value:.2f}")
print(f"Lucro acumulado: {profit:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

print("\nTrade Log:")
for trade in trade_log:
    print(trade)

Valor final da carteira: 10166.41
Lucro acumulado: 166.41
Sharpe Ratio: 3.97

Trade Log:
{'step': 1, 'asset': 'PETR4', 'action': 'Sell', 'price': 27.92, 'cash_before': 10000.0, 'positions_before': {'BRFS3': 0, 'PETR4': 0, 'VALE3': 0}}
{'step': 1, 'asset': 'VALE3', 'action': 'Buy', 'price': 83.16, 'cash_before': 10000.0, 'positions_before': {'BRFS3': 0, 'PETR4': 0, 'VALE3': 0}}
{'step': 2, 'asset': 'PETR4', 'action': 'Buy', 'price': 28.6, 'cash_before': 9916.75684, 'positions_before': {'BRFS3': 0, 'PETR4': 0, 'VALE3': 1}}
{'step': 2, 'asset': 'VALE3', 'action': 'Sell', 'price': 84.39, 'cash_before': 9916.75684, 'positions_before': {'BRFS3': 0, 'PETR4': 0, 'VALE3': 1}}
{'step': 3, 'asset': 'PETR4', 'action': 'Buy', 'price': 29.53, 'cash_before': 9972.43385, 'positions_before': {'BRFS3': 0, 'PETR4': 1, 'VALE3': 0}}
{'step': 3, 'asset': 'VALE3', 'action': 'Buy', 'price': 85.73, 'cash_before': 9972.43385, 'positions_before': {'BRFS3': 0, 'PETR4': 1, 'VALE3': 0}}
{'step': 4, 'asset': 'VALE3'